**Writing Smart Contracts 2024: Final Project by Kristina Odermatt (odermk@usi.ch)**

**(6) Voting**:  A club has a membership list with the public keys of all members. Write a smart contract that allows members to vote for one of three candidates (Rossi, Smith or Meier). Extension: (1) Ensure that voting is only possible in a certain time frame.




# **Notebook 3: Testing the smart contract**

## Setup

In [1]:
# loading shared code and credentials
from algo_util import *

cred = load_credentials("./credentials_project")
members = cred['Members']
club = cred['Club']


In [2]:
# importing necessary modules and functions from the Algorand SDK
from algosdk import account, mnemonic
from algosdk.v2client import algod
from algosdk import transaction
from algosdk.transaction import PaymentTxn
from algosdk.transaction import AssetConfigTxn, AssetTransferTxn, AssetFreezeTxn, ApplicationNoOpTxn 
from algosdk.transaction import LogicSig, LogicSigTransaction

import algosdk.error
import json
import base64
import datetime
import hashlib


In [3]:
# importing all the contents from the pyteal module
from pyteal import *

In [4]:
# Initialize the algod client (Testnet)
algod_client = algod.AlgodClient(algod_token='', algod_address=cred['algod_test'], headers=cred['api_token'])
algod_client.status()['last-round']

38927160

In [5]:
# define the app_id from '01_Create_SC'

app_id = 640731779             # <--- ‼️‼️ INSERT/UPDATE APP_ID

## 3.1 Voting for a non existing candidate

In [6]:
member_name = 'Member_15' 

candidate_name = "Ferrari" # <-- non existing candidate


# Step 1: Prepare the transaction
user = members[member_name]  
sp = algod_client.suggested_params()
txn = transaction.ApplicationNoOpTxn(
    user['public'],  
    sp,
    app_id,
    [bytes(candidate_name, 'utf-8')]  
)

# Step 2: Sign the transaction
stxn = txn.sign(user['private'])

# Step 3: Send the transaction
txid = algod_client.send_transactions([stxn])

# Step 4: Wait for confirmation
txinfo = wait_for_confirmation(algod_client, txid)



AlgodHTTPError: TransactionPool.Remember: transaction C7PWQVKAZNF37H6XRMVH6Q3AFEAIUYHHJVGZXSLBG3ZTLMEMUZ7A: transaction rejected by ApprovalProgram

## 3.2 Club user tries to cast a vote

In [7]:
user = club  

# Step 1: Prepare the transaction
sp = algod_client.suggested_params()
candidate_name = "Meier"  
txn = transaction.ApplicationNoOpTxn(
    user['public'],  
    sp,
    app_id,
    [bytes(candidate_name, 'utf-8')]  
)

# Step 2: Sign the transaction
stxn = txn.sign(user['private'])

# Step 3: Send the transaction
txid = algod_client.send_transactions([stxn])

# Step 4: Wait for confirmation
txinfo = wait_for_confirmation(algod_client, txid)


AlgodHTTPError: TransactionPool.Remember: transaction PNWQZFHKEPEJ2Z27HIOSTIF5TACRC5FUYTEX4JSDQ7OGOK6CACTA: logic eval error: cannot fetch key, KPH4VDOVVVP3BWOUJYIETXGDMS3ZQR3DHCQ7HBEHTVCKIN264VW7WUJZ5E has not opted in to app 640731779. Details: app=640731779, pc=154, opcodes=intc_0 // 0; bytec_0 // "has_voted"; app_local_get

## 3.3 Double voting

In [8]:
member_name = 'Member_1' 

candidate_name = "Meier" 

# Step 1: Prepare the transaction
user = members[member_name]  
sp = algod_client.suggested_params()
txn = transaction.ApplicationNoOpTxn(
    user['public'],  # Sender's public key
    sp,
    app_id,
    [bytes(candidate_name, 'utf-8')]  # Include the candidate's name as an argument
)

# Step 2: Sign the transaction
stxn = txn.sign(user['private'])


# Step 3: Send the transaction
txid = algod_client.send_transactions([stxn])

# Step 4: Wait for confirmation
txinfo = wait_for_confirmation(algod_client, txid)



AlgodHTTPError: TransactionPool.Remember: transaction MKVDAGRR56NXGSY5Q6Q6IQFRAYOCHXO2YRLGVHEQP4JZY5MK6KEA: transaction rejected by ApprovalProgram

## 3.4 A member tries to vote after voting period has ended
 ‼️Before testing 3.4 check that the voting period has endend in order to have an error


In [9]:
# checking voting period
algod_client.status()['last-round']

38927556

In [10]:
# compare last_round to EndRound. When EndRound < last-round, proceed with testing
read_global_state(algod_client,app_id)

{'EndRound': 38927305,
 'StartRound': 38927105,
 'Rossi': 7,
 'Meier': 4,
 'Smith': 3,
 'Members': 15,
 'Note': 'Welcome to the Voting System',
 'TotalVotes': 14}

In [11]:
member_name = 'Member_15' 

candidate_name = "Rossi" 


# Step 1: Prepare the transaction
user = members[member_name]  
sp = algod_client.suggested_params()
txn = transaction.ApplicationNoOpTxn(
    user['public'],  # Sender's public key
    sp,
    app_id,
    [bytes(candidate_name, 'utf-8')]  # Include the candidate's name as an argument
)

# Step 2: Sign the transaction
stxn = txn.sign(user['private'])

# Step 3: Send the transaction
txid = algod_client.send_transactions([stxn])

# Step 4: Wait for confirmation
txinfo = wait_for_confirmation(algod_client, txid)




AlgodHTTPError: TransactionPool.Remember: transaction UDNS2QJYEA2ZSHESFH56DGDZSFMDRQMACZWQZXCLR5TSJS4YCWYA: transaction rejected by ApprovalProgram

## 3.5 A non-creator tries delete the App

In [12]:
creator = members['Member_15']

# Step 1: Prepare transaction
sp = algod_client.suggested_params()
txn = transaction.ApplicationDeleteTxn(creator['public'], sp, app_id)

# Step 2: sign
stxn = txn.sign(creator['private'])

# Step 3: send
txid = algod_client.send_transactions([stxn])

# Step4 : wait for confirmation
txinfo = wait_for_confirmation(algod_client, txid)

# display results
print("Deleted app-id:", txinfo["txn"]["txn"]["apid"])


AlgodHTTPError: TransactionPool.Remember: transaction 4VUZS4MGNAKJ76II3ZKIRPAPLATIAWG67IEYWMZ3VI7HYUYDB5WQ: transaction rejected by ApprovalProgram